In [1]:
%autoreload 2
%load_ext autoreload

ERROR:root:Line magic function `%autoreload` not found.


In [2]:
import theano
import theano.tensor as T
import numpy as np 
import matplotlib.pyplot as plt 

In [3]:
from sklearn.utils import shuffle
from util import init_weight, get_robert_frost

In [8]:
class SimpleRNN:
    def __init__(self, D, M, V):
        self.D = D # dimensionality of word embedding
        self.M = M # hidden layer size
        self.V = V # vocabulary size

    def fit(self, X, learning_rate=10e-1, mu=0.99, reg=1.0, activation=T.tanh, epochs=500, show_fig=False):
        N = len(X)
        D = self.D
        M = self.M
        V = self.V
        self.f = activation

        # initial weights
        We = init_weight(V, D)
        
        # r unit 
        Wxr = init_weight(D, M)
        Whr = init_weight(M, M)
        br  = np.zeros(M)
    
        # z unit 
        Wxz = init_weight(D, M)
        Whz = init_weight(M, M)
        bz  = np.zeros(M)
        
        # h hat 
        Wx = init_weight(D, M)
        Wh = init_weight(M, M)
        bh = np.zeros(M)
        h0 = np.zeros(M)
        
        # Rated Unit 
        Wo = init_weight(M, V)
        bo = np.zeros(V)
        
        thX, thY, py_x, prediction = self.set(We, Wxr, Whr, Wxz, Whz, Wx, Wh, br, bz, bh, h0, Wo, bo, activation)

        lr = T.scalar('lr')

        cost = -T.mean(T.log(py_x[T.arange(thY.shape[0]), thY]))
        grads = T.grad(cost, self.params)
        dparams = [theano.shared(p.get_value()*0) for p in self.params]

        updates = [
            (p, p + mu*dp - learning_rate*g) for p, dp, g in zip(self.params, dparams, grads)
        ] + [
            (dp, mu*dp - learning_rate*g) for dp, g in zip(dparams, grads)
        ]

        self.predict_op = theano.function(inputs=[thX], outputs=prediction)
        self.train_op = theano.function(
            inputs=[thX, thY],
            outputs=[cost, prediction],
            updates=updates
        )

        costs = []
        for i in xrange(epochs):
            X = shuffle(X)
            n_correct = 0
            n_total = 0 
            cost = 0
            for j in xrange(N):
                # problem! many words --> END token are overrepresented
                # result: generated lines will be very short
                # we will try to fix in a later iteration
                # BAD! magic numbers 0 and 1...
                if np.random.random() < 0.1:
                    input_sequence = [0] + X[j]
                    output_sequence = X[j] + [1]
                else:
                    input_sequence = [0] + X[j][:-1]
                    output_sequence = X[j]
                    
                n_total += len(output_sequence)
                    
                # we set 0 to start and 1 to end
                c, p = self.train_op(input_sequence, output_sequence)
                # print "p:", p
                cost += c
                # print "j:", j, "c:", c/len(X[j]+1)
                for pj, xj in zip(p, output_sequence):
                    if pj == xj:
                        n_correct += 1
            print "i:", i, "cost:", cost, "correct rate:", (float(n_correct)/n_total)
            costs.append(cost)

        if show_fig:
            plt.plot(costs)
            plt.show()
    
    def save(self, filename):
        np.savez(filename, *[p.get_value() for p in self.params])

    @staticmethod
    def load(filename, activation):
        # TODO: would prefer to save activation to file too
        #  We, Wxr, Whr, Wxz, Whz, Wx, Wh, br, bz, bh, h0, Z,  Wo, bo
        
        npz = np.load(filename)
        We = npz['arr_0']
        Wxr = npz['arr_1']
        Whr = npz['arr_2']
        Wxz = npz['arr_3']
        Whz = npz['arr_4']
        Wx = npz['arr_5']
        Wh = npz['arr_6']
        br = npz['arr_7']
        bz = npz['arr_8']
        bh = npz['arr_9']
        h0 = npz['arr_10']
        Wo = npz['arr_11']
        bo = npz['arr_12']
        
        V, D = We.shape
        _, M = Wx.shape
        rnn = SimpleRNN(D, M, V)
        rnn.set(We, Wxr, Whr, Wxz, Whz, Wx, Wh, br, bz, bh, h0, Wo, bo, activation)
        return rnn

    def set(self, We, Wxr, Whr, Wxz, Whz, Wx, Wh, br, bz, bh, h0, Wo, bo, activation):
        #  We, Wxr, Whr, Wxz, Whz, Wx, Wh, br, bz, bh, h0, Wo, bo
        
        self.f = activation

        # redundant - see how you can improve it
        self.We = theano.shared(We)
        self.Wxr = theano.shared(Wxr)
        self.Whr = theano.shared(Whr)
        self.Wxz = theano.shared(Wxz)
        self.Whz = theano.shared(Whz)
        self.Wx = theano.shared(Wx)
        self.Wh = theano.shared(Wh)
        self.br = theano.shared(br)
        self.bz = theano.shared(bz)
        self.bh = theano.shared(bh)
        self.h0 = theano.shared(h0)
        self.Wo = theano.shared(Wo)
        self.bo = theano.shared(bo)
        
        self.params = [self.We, self.Wxr, self.Whr, self.Wxz, self.Whz, self.Wx, self.Wh, self.br, self.bz, \
                       self.bh, self.h0, self.Wo, self.bo]

        thX = T.ivector('X')
        Ei = self.We[thX] # will be a TxD matrix
        thY = T.ivector('Y')

        def recurrence(x_t, h_t1):
            # returns h(t), y(t)
            r_t = T.nnet.nnet.sigmoid(x_t.dot(self.Wxr) + h_t1.dot(self.Whr) + self.br)
            z_t = T.nnet.nnet.sigmoid(x_t.dot(self.Wxz) + h_t1.dot(self.Whz) + self.bz)
            hhat_t = self.f(x_t.dot(self.Wx) + (r_t * h_t1).dot(self.Wh) + self.bh)
            h_t = ( 1 - z_t) * h_t1 + z_t * hhat_t
            y_t = T.nnet.softmax(h_t.dot(self.Wo) + self.bo)
            return h_t, y_t

        [h, y], _ = theano.scan(
            fn=recurrence,
            outputs_info=[self.h0, None],
            sequences=Ei,
            n_steps=Ei.shape[0],
        )

        py_x = y[:, 0, :]
        prediction = T.argmax(py_x, axis=1)
        self.predict_op = theano.function(
            inputs=[thX],
            outputs=[py_x, prediction],
            allow_input_downcast=True,
        )
        return thX, thY, py_x, prediction 

    def generate(self, word2idx):
        # convert word2idx -> idx2word
        idx2word = {v:k for k,v in word2idx.iteritems()}
        V = len(word2idx)

        # generate 4 lines at a time
        n_lines = 0

        # why? because using the START symbol will always yield the same first word!
        X = [ 0 ]
        while n_lines < 4:
            # print "X:", X
            PY_X, _ = self.predict_op(X)
            PY_X = PY_X[-1].flatten()
            P = [ np.random.choice(V, p=PY_X)]
            X = np.concatenate([X, P]) # append to the sequence
            # print "P.shape:", P.shape, "P:", P
            P = P[-1] # just grab the most recent prediction
            if P > 1:
                # it's a real word, not start/end token
                word = idx2word[P]
                print word,
            elif P == 1:
                # end token
                n_lines += 1
                X = [0]
                print ''
                


def train_poetry():
    # students: tanh didn't work but you should try it
    sentences, word2idx = get_robert_frost()
    rnn = SimpleRNN(50, 50, len(word2idx))
    rnn.fit(sentences, learning_rate=10e-5, show_fig=True, activation=T.nnet.relu, epochs=2000)
    rnn.save('RRNN_D50_M50_epochs2000_relu.npz')

def generate_poetry():
    sentences, word2idx = get_robert_frost()
    rnn = SimpleRNN.load('RRNN_D50_M50_epochs2000_relu.npz', T.nnet.relu)
    rnn.generate(word2idx)

def wikipedia():
    sentences, word2idx = get_wikipedia_data()
    print "finished retrieving data"
    print "vocab size:", len(word2idx), "number of sentences:", len(sentences)
    rnn = SimpleRNN(20, 15, len(word2idx))
    rnn.fit(sentences, learning_rate=10e-5, show_fig=True, activation=T.nnet.relu)

#if __name__ == '__main__':
#    train_poetry()
#    generate_poetry()
#    #wikipedia()



i: 0 cost: 10985.0063655 correct rate: 0.0489671707857
i: 1 cost: 10828.5938322 correct rate: 0.0502803566504
i: 2 cost: 10642.4964695 correct rate: 0.0503729625196
i: 3 cost: 10157.3667875 correct rate: 0.0503537627492
i: 4 cost: 9503.92312365 correct rate: 0.050363686585
i: 5 cost: 9174.65335494 correct rate: 0.0497604128271
i: 6 cost: 9028.78902634 correct rate: 0.0506573503723
i: 7 cost: 8970.18939751 correct rate: 0.0535960954047
i: 8 cost: 8924.10342382 correct rate: 0.0551311550851
i: 9 cost: 8886.02784685 correct rate: 0.0552796173657
i: 10 cost: 8854.18870003 correct rate: 0.056956201693
i: 11 cost: 8832.85806974 correct rate: 0.0562304435855
i: 12 cost: 8807.22065434 correct rate: 0.0577524370057
i: 13 cost: 8788.58440627 correct rate: 0.0566350492679
i: 14 cost: 8757.34749463 correct rate: 0.0592947242427
i: 15 cost: 8739.35677669 correct rate: 0.0544800884956
i: 16 cost: 8705.95694982 correct rate: 0.0600846521899
i: 17 cost: 8682.60628037 correct rate: 0.0567584999539
i: 1

In [9]:
train_poetry()

In [10]:
generate_poetry()

what it is to me will 
never fell and i 
and me on the floor 
it in creation 


In [11]:
generate_poetry()

i stay to say 
excuse me if i was too and 
sitting or standing as he chose 
the utmost star 


In [12]:
generate_poetry()

today 
two miles it was to his abode 
i cant explain it any other way tended 
john hall touch me not if the night stark do 
